In [70]:
import pandas as pd
import numpy as np
from datetime import datetime                                                                                             
                                                                           
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [ ]:
# train_data.reset_index().drop('index',axis=1).head()
# df = test_data
# df['h_tweets'] = df['home_tweets'].apply(len)
# df['a_tweets'] = df['away_tweets'].apply(len)
# h_tweets = df['h_tweets'].mean()
# a_tweets = df['a_tweets'].mean()
# print(h_tweets)
# print(a_tweets)
# len(test_data[test_data.HOME_TEAM_WINS==0])
# train_data.reset_index().drop(['index','HOME_TEAM_ID','TEAM_ID_away','tweets'],axis=1).head()
# for game in train_data['tweets']:
#     print(game[0])
# print(train_data['tweets'][0])

In [71]:
train_data = pd.read_parquet("data/train.pq", engine='fastparquet')
test_data = pd.read_parquet("data/test.pq", engine='fastparquet')

In [94]:
train_home_eb = np.load("data/train_home_embeddings.npy")
train_away_eb = np.load("data/train_away_embeddings.npy")
test_home_eb = np.load("data/test_home_embeddings.npy")
test_away_eb = np.load("data/test_away_embeddings.npy")
# print(train_home_eb[[0,1,2]])
# print(train_away_eb[[0,1,2]])
# print(test_home_eb[[0,1,2]])
# print(test_away_eb[[0,1,2]])
# print(np.allclose(train_home_eb, train_away_eb))
# print(np.allclose(test_home_eb, test_away_eb))
# print(np.allclose(train_home_eb[:807],test_home_eb))
# print(np.allclose(train_away_eb[:807],test_away_eb))
# train_home_eb.shape

(3228, 768)

In [95]:
# train
# trheb = train_home_eb.reshape(train_home_eb.shape[0], 1, -1)
# traeb = train_away_eb.reshape(train_away_eb.shape[0], 1, -1)
# TRAIN_X = np.concatenate(train_home_eb, train_away_eb)
TRAIN_X = np.add(train_home_eb, train_away_eb)
# TRAIN_X = TRAIN_X.reshape(TRAIN_X.shape[0], -1)
TRAIN_Y = np.array(train_data['HOME_TEAM_WINS'])

# test
# tsheb = test_home_eb.reshape(test_home_eb.shape[0], 1, -1)
# tsaeb = test_away_eb.reshape(test_away_eb.shape[0], 1, -1)
# test_X = np.concatenate((tsheb,tsaeb), axis=1)
TEST_X = np.add(test_home_eb, test_away_eb)
# TEST_X = TEST_X.reshape(TEST_X.shape[0], -1)
TEST_Y = np.array(test_data['HOME_TEAM_WINS'])

In [99]:
param_test = {
    'criterion':['gini','entropy'],
    'max_depth':range(1,2,3),
#     'min_impurity_decrease':[6],
    'class_weight':[None,'balanced']
}
gsearch = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid = param_test,
    scoring='accuracy',
    n_jobs=5,
    cv=5
)
gsearch.fit(TRAIN_X,TRAIN_Y)
print(gsearch.best_params_, gsearch.best_score_)

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 1} 0.5814745482036144


In [105]:
# classifiers = [
#     DummyClassifier(strategy="uniform", random_state=0),
#     LogisticRegression(),
#     KNeighborsClassifier(n_neighbors=10, weights='distance'),
#     SVC(kernel='rbf'),
#     DecisionTreeClassifier(),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GradientBoostingClassifier(),
#     GaussianNB(),
#     Lasso()
# ]

def single_run(X_train, y_train, X_test, y_test):
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr,tpr)
    return auc, clf


skf = StratifiedKFold(n_splits=5)
# name = clf.__class__.__name__
best_auc = 0
best_model = None
aucs = []

for train_index, test_index in skf.split(TRAIN_X, TRAIN_Y):
    X_train, y_train = TRAIN_X[train_index], TRAIN_Y[train_index]
    X_test, y_test = TRAIN_X[test_index], TRAIN_Y[test_index]

    auc, curr_model = single_run(X_train, y_train, X_test, y_test)
    aucs.append(auc)
    if auc > best_auc:
        best_auc = auc
        best_model = curr_model

test_pred = best_model.predict(TEST_X)
fpr, tpr, _ = metrics.roc_curve(TEST_Y, test_pred)
test_auc = metrics.auc(fpr,tpr)
f1_score = metrics.f1_score(TEST_Y, test_pred, average="weighted")
print(f"GradientBoostingClassifier:\n"
      f" - train {round(np.mean(aucs),3)} ({round(np.std(aucs),3)})\n"
      f" - test {round(test_auc,3)}\n"
      f" - f1 {round(f1_score, 3)}")

DecisionTree:
 - train 0.515 (0.012)
 - test 0.515
 - f1 0.518
